# Mini Proyecto

datos: https://www.cs.cmu.edu/~ark/personas/

In [10]:
# Librerias

import pandas as pd
import pyspark
# Cargar Funciones Extra
from pyspark.sql.functions import * 
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Proyecto_1').getOrCreate()

In [11]:
#Carga de bibliotecas extras
import numpy as np
import matplotlib.pyplot as plt

#Carga de NLTK
import nltk
import nltk
nltk.download('stopwords')
import string

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/danielfragoso/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Generación de Funciones auxiliares para limpiar algunos registros que se utilizaran posteriormente.

In [12]:
# Lectura de datos TSV
def read_tsv(direccion):
    import csv
    tsv_file = open(direccion)
    read_tsv = csv.reader(tsv_file, delimiter= "\t")
    lista = [row for row in read_tsv]
    tsv_file.close()
    return(spark.sparkContext.parallelize(lista))

# Lectura de datos TXT
def read_txt(direccion):
    import csv
    txt_file = open(direccion)
    read_tsv = csv.reader(txt_file, delimiter= "\t")
    lista = [row for row in read_tsv]
    txt_file.close()
    return(spark.sparkContext.parallelize(lista))

In [13]:
# Importamos datos:

character = read_tsv('MovieSummaries/character.metadata.tsv')
movie = read_tsv('MovieSummaries/movie.metadata.tsv')

In [14]:
name = read_txt('MovieSummaries/name.clusters.txt')
tvtropes = read_txt('MovieSummaries/tvtropes.clusters.txt')
plot_summaries =read_txt('MovieSummaries/plot_summaries.txt')

## ¿Cual es género más visto?

In [15]:
# Función que divide los géneros.

def sep_gen(renglon):
    renglon_separado = renglon[8]
    import re
    generos = re.findall(r'"([\w\s]+)"', renglon_separado)
    for g in generos:
        yield g

In [16]:
genere = movie.flatMap(sep_gen)
genere = genere.map(lambda renglon: (renglon, 1))
# Conteo
genere_count = genere.reduceByKey(lambda val_1 , val_2: val_1 + val_2).sortBy(lambda renglon: renglon[1], ascending = False)

22/02/28 22:30:19 WARN TaskSetManager: Stage 0 contains a task of very large size (4397 KiB). The maximum recommended task size is 1000 KiB.


In [17]:
genere_count.take(10)

[('Drama', 34007),
 ('Comedy', 16349),
 ('Romance Film', 10234),
 ('Action', 8798),
 ('Thriller', 8744),
 ('Short Film', 8141),
 ('World cinema', 7155),
 ('Crime Fiction', 6948),
 ('Indie', 6897),
 ('Documentary', 5630)]

## La palabra más larga de las descripciones




In [18]:
# Comparación de Strings
def com_string(x, y):
    if len(x) > len(y):
        return x
    return y
  
#Eliminacion de url's
def del_url(renglon):
    import re
    generos = re.sub(r'http\S+', ' ', renglon)
    return generos

In [19]:
desc = plot_summaries.map(lambda x: x[1])
desc = desc.map(del_url)

In [20]:
# Reduzcammos a palabras

words = desc.map(lambda x: x.split(' '))

#Eliminamos signos de puntuación y espacios en blanco
import string
def removePunctuationsFunct(x):
    list_punct=list(string.punctuation)
    filtered = [''.join(c for c in s if c not in list_punct) for s in x] 
    filtered_space = [s for s in filtered if s] #remove empty space 
    return filtered

#Removemos stopwords (palabras vacias)
def removeStopWordsFunct(x):
    from nltk.corpus import stopwords
    stop_words=set(stopwords.words('english'))
    filteredSentence = [w for w in x if not w in stop_words]
    return filteredSentence

words = words.map(removeStopWordsFunct)
words = words.map(removePunctuationsFunct)


In [21]:
def sep(renglon):
    for g in renglon:
        return g

words_sep = words.map(sep)

In [22]:
word_legth = words_sep.reduce(com_string)

22/02/28 22:30:40 WARN TaskSetManager: Stage 7 contains a task of very large size (18273 KiB). The maximum recommended task size is 1000 KiB.


In [23]:
word_legth

'PlotdatePrologueThursdayMondaySaturdayTuesdayFridaySundayWednesdayEpilogue'

## Contar el número de veces que aparece la palabra 'war' en la descripción 

In [24]:
cont = plot_summaries.map(lambda x: (x[0], x[1].split(' ')))
cont = cont.map(lambda x: (x[0], removeStopWordsFunct(x[1])))
cont = cont.map(lambda x: (x[0], removePunctuationsFunct(x[1])))

In [25]:
cont_war = cont.map(lambda x: (x[0], x[1].count('war')))

In [26]:
cont_war = cont_war.sortBy(lambda renglon: renglon[1], ascending = False)

22/02/28 22:32:52 WARN TaskSetManager: Stage 8 contains a task of very large size (18273 KiB). The maximum recommended task size is 1000 KiB.
22/02/28 22:34:29 WARN TaskSetManager: Stage 9 contains a task of very large size (18273 KiB). The maximum recommended task size is 1000 KiB.


In [27]:
cont_war.take(5)

22/02/28 22:35:49 WARN TaskSetManager: Stage 10 contains a task of very large size (18273 KiB). The maximum recommended task size is 1000 KiB.


[('886892', 10),
 ('3843721', 9),
 ('3653120', 8),
 ('3903364', 8),
 ('1432805', 8)]

In [28]:
war_movies = cont_war.filter(lambda renglon: renglon[1] > 5)

## ¿Cual es la película que más menciona 'war' en su descripción?

In [29]:
movie.filter(lambda x: x[0] == '886892').take(1)

22/02/28 22:37:16 WARN TaskSetManager: Stage 12 contains a task of very large size (4397 KiB). The maximum recommended task size is 1000 KiB.


[['886892',
  '/m/03lv4x',
  'Breaker Morant',
  '1980',
  '4735000',
  '107.0',
  '{"/m/02h40lc": "English Language"}',
  '{"/m/0chghy": "Australia", "/m/0ctw_b": "New Zealand"}',
  '{"/m/01fc50": "Anti-war", "/m/0hj3mx2": "British Empire Film", "/m/03g3w": "History", "/m/04xvlr": "Period piece", "/m/07s9rl0": "Drama", "/m/082gq": "War film", "/m/07k67c": "Anti-war film", "/m/05bh16v": "Courtroom Drama"}']]

<img src='MovieSummaries/Breaker_morant.png'>

## ¿De que país provienen principalmente las películas de guerra?

In [30]:
war_movies.take(5)

[('886892', 10),
 ('3843721', 9),
 ('3653120', 8),
 ('3903364', 8),
 ('1432805', 8)]

In [31]:
country = movie.map(lambda x: (x[0], x[7]))
country.take(1)

22/02/28 22:37:17 WARN TaskSetManager: Stage 15 contains a task of very large size (4397 KiB). The maximum recommended task size is 1000 KiB.


[('975900', '{"/m/09c7w0": "United States of America"}')]

In [32]:
country_war = country.join(war_movies)

In [33]:
def sep_gen_2(renglon):
    renglon_separado = renglon[0]
    import re
    generos = re.findall(r'"([\w\s]+)"', renglon_separado)
    for g in generos:
        yield g

In [34]:
country_war = country_war.flatMap(lambda x: sep_gen_2(x[1]))

In [35]:
country_war_cont = country_war.map(lambda x: (x, 1)).reduceByKey(lambda val_1 , val_2: val_1 + val_2).sortBy(lambda renglon: renglon[1], ascending = False)

22/02/28 22:37:20 WARN TaskSetManager: Stage 17 contains a task of very large size (4397 KiB). The maximum recommended task size is 1000 KiB.


In [36]:
country_war_cont.take(10)

[('United States of America', 12),
 ('United Kingdom', 6),
 ('Japan', 4),
 ('Canada', 2),
 ('Serbia', 2),
 ('Bosnia and Herzegovina', 2),
 ('Australia', 2),
 ('France', 2),
 ('Croatia', 2),
 ('Argentina', 1)]

## ¿Qué palabras se repiten más?


In [37]:
# Visualización de RDD con datos
for letra in plot_summaries.take(5):
  print(letra)

['23890098', "Shlykov, a hard-working taxi driver and Lyosha, a saxophonist, develop a bizarre love-hate relationship, and despite their prejudices, realize they aren't so different after all."]
['31186339', 'The nation of Panem consists of a wealthy Capitol and twelve poorer districts. As punishment for a past rebellion, each district must provide a boy and girl  between the ages of 12 and 18 selected by lottery  for the annual Hunger Games. The tributes must fight to the death in an arena; the sole survivor is rewarded with fame and wealth. In her first Reaping, 12-year-old Primrose Everdeen is chosen from District 12. Her older sister Katniss volunteers to take her place. Peeta Mellark, a baker\'s son who once gave Katniss bread when she was starving, is the other District 12 tribute. Katniss and Peeta are taken to the Capitol, accompanied by their frequently drunk mentor, past victor Haymitch Abernathy. He warns them about the "Career" tributes who train intensively at special acad

22/02/28 22:37:25 WARN TaskSetManager: Stage 39 contains a task of very large size (18273 KiB). The maximum recommended task size is 1000 KiB.


In [38]:
# Transformar textos a minúsculas
plot_summaries_minus = plot_summaries.map(lambda x: (x[1].lower()))

for letra in plot_summaries_minus.take(5):
  print(letra)

shlykov, a hard-working taxi driver and lyosha, a saxophonist, develop a bizarre love-hate relationship, and despite their prejudices, realize they aren't so different after all.
the nation of panem consists of a wealthy capitol and twelve poorer districts. as punishment for a past rebellion, each district must provide a boy and girl  between the ages of 12 and 18 selected by lottery  for the annual hunger games. the tributes must fight to the death in an arena; the sole survivor is rewarded with fame and wealth. in her first reaping, 12-year-old primrose everdeen is chosen from district 12. her older sister katniss volunteers to take her place. peeta mellark, a baker's son who once gave katniss bread when she was starving, is the other district 12 tribute. katniss and peeta are taken to the capitol, accompanied by their frequently drunk mentor, past victor haymitch abernathy. he warns them about the "career" tributes who train intensively at special academies and almost always win. du

22/02/28 22:37:26 WARN TaskSetManager: Stage 40 contains a task of very large size (18273 KiB). The maximum recommended task size is 1000 KiB.


In [39]:
# Separar texto por palabra
plot_summaries_sep = plot_summaries_minus.map(lambda x: (x.split(' ')))

for letra in plot_summaries_sep.take(5):
  print(letra)

['shlykov,', 'a', 'hard-working', 'taxi', 'driver', 'and', 'lyosha,', 'a', 'saxophonist,', 'develop', 'a', 'bizarre', 'love-hate', 'relationship,', 'and', 'despite', 'their', 'prejudices,', 'realize', 'they', "aren't", 'so', 'different', 'after', 'all.']
['the', 'nation', 'of', 'panem', 'consists', 'of', 'a', 'wealthy', 'capitol', 'and', 'twelve', 'poorer', 'districts.', 'as', 'punishment', 'for', 'a', 'past', 'rebellion,', 'each', 'district', 'must', 'provide', 'a', 'boy', 'and', 'girl', '', 'between', 'the', 'ages', 'of', '12', 'and', '18', 'selected', 'by', 'lottery', '', 'for', 'the', 'annual', 'hunger', 'games.', 'the', 'tributes', 'must', 'fight', 'to', 'the', 'death', 'in', 'an', 'arena;', 'the', 'sole', 'survivor', 'is', 'rewarded', 'with', 'fame', 'and', 'wealth.', 'in', 'her', 'first', 'reaping,', '12-year-old', 'primrose', 'everdeen', 'is', 'chosen', 'from', 'district', '12.', 'her', 'older', 'sister', 'katniss', 'volunteers', 'to', 'take', 'her', 'place.', 'peeta', 'mellark

22/02/28 22:37:26 WARN TaskSetManager: Stage 41 contains a task of very large size (18273 KiB). The maximum recommended task size is 1000 KiB.


In [40]:
# Quitar las stop words

def removeStopWordsFunct(x):
    from nltk.corpus import stopwords
    stop_words=set(stopwords.words('english'))
    filteredSentence = [w for w in x if not w in stop_words]
    return filteredSentence

plot_summaries_stop = plot_summaries_sep.map(removeStopWordsFunct)

for palabra in plot_summaries_stop.take(5):
  print(palabra)

22/02/28 22:37:27 WARN TaskSetManager: Stage 42 contains a task of very large size (18273 KiB). The maximum recommended task size is 1000 KiB.


['shlykov,', 'hard-working', 'taxi', 'driver', 'lyosha,', 'saxophonist,', 'develop', 'bizarre', 'love-hate', 'relationship,', 'despite', 'prejudices,', 'realize', 'different', 'all.']
['nation', 'panem', 'consists', 'wealthy', 'capitol', 'twelve', 'poorer', 'districts.', 'punishment', 'past', 'rebellion,', 'district', 'must', 'provide', 'boy', 'girl', '', 'ages', '12', '18', 'selected', 'lottery', '', 'annual', 'hunger', 'games.', 'tributes', 'must', 'fight', 'death', 'arena;', 'sole', 'survivor', 'rewarded', 'fame', 'wealth.', 'first', 'reaping,', '12-year-old', 'primrose', 'everdeen', 'chosen', 'district', '12.', 'older', 'sister', 'katniss', 'volunteers', 'take', 'place.', 'peeta', 'mellark,', "baker's", 'son', 'gave', 'katniss', 'bread', 'starving,', 'district', '12', 'tribute.', 'katniss', 'peeta', 'taken', 'capitol,', 'accompanied', 'frequently', 'drunk', 'mentor,', 'past', 'victor', 'haymitch', 'abernathy.', 'warns', '"career"', 'tributes', 'train', 'intensively', 'special', 'ac

In [41]:
# Quitar los signos de puntuación 

import string
def removePunctuationsFunct(x):
    list_punct=list(string.punctuation)
    filtered = [''.join(c for c in s if c not in list_punct) for s in x] 
    filtered_space = [s for s in filtered if s] #remove empty space 
    return filtered

plot_summaries_punct = plot_summaries_stop.map(removePunctuationsFunct)

for palabra in plot_summaries_punct.take(5):
  print(palabra)

22/02/28 22:37:32 WARN TaskSetManager: Stage 43 contains a task of very large size (18273 KiB). The maximum recommended task size is 1000 KiB.


['shlykov', 'hardworking', 'taxi', 'driver', 'lyosha', 'saxophonist', 'develop', 'bizarre', 'lovehate', 'relationship', 'despite', 'prejudices', 'realize', 'different', 'all']
['nation', 'panem', 'consists', 'wealthy', 'capitol', 'twelve', 'poorer', 'districts', 'punishment', 'past', 'rebellion', 'district', 'must', 'provide', 'boy', 'girl', '', 'ages', '12', '18', 'selected', 'lottery', '', 'annual', 'hunger', 'games', 'tributes', 'must', 'fight', 'death', 'arena', 'sole', 'survivor', 'rewarded', 'fame', 'wealth', 'first', 'reaping', '12yearold', 'primrose', 'everdeen', 'chosen', 'district', '12', 'older', 'sister', 'katniss', 'volunteers', 'take', 'place', 'peeta', 'mellark', 'bakers', 'son', 'gave', 'katniss', 'bread', 'starving', 'district', '12', 'tribute', 'katniss', 'peeta', 'taken', 'capitol', 'accompanied', 'frequently', 'drunk', 'mentor', 'past', 'victor', 'haymitch', 'abernathy', 'warns', 'career', 'tributes', 'train', 'intensively', 'special', 'academies', 'almost', 'always

In [42]:
# Aplanar las palabras

def unir(lista):
  todos = []
  for i in range(len(lista)):
    for j in range(len(lista[i])):
      todos.append(lista[i][j])
  return lista

plot_summaries_flat = plot_summaries_punct.flatMap(unir)

for palabra in plot_summaries_flat.take(10):
  print(palabra)

22/02/28 22:37:35 WARN TaskSetManager: Stage 44 contains a task of very large size (18273 KiB). The maximum recommended task size is 1000 KiB.


shlykov
hardworking
taxi
driver
lyosha
saxophonist
develop
bizarre
lovehate
relationship


In [43]:
# Contar ocurrencias de las palabras

plot_summaries_palabras = plot_summaries_flat.map(lambda x: (x,1)) \
                                              .reduceByKey(lambda a,b: a+b) \
                                              .sortBy(lambda x: x[1], ascending = False)

for palabra in plot_summaries_palabras.take(10):
  print(palabra)

22/02/28 22:37:38 WARN TaskSetManager: Stage 45 contains a task of very large size (18273 KiB). The maximum recommended task size is 1000 KiB.


('', 142895)
('one', 31215)
('him', 27672)
('back', 24460)
('two', 21753)
('film', 20271)
('tells', 19109)
('her', 18468)
('father', 18338)
('man', 18170)


## Q-Gramas

In [44]:
# Función para sacar q-gramas
def gramas(palabra, q):
  n = len(palabra)
  lista_gramas = []
  for i in range(n-q+1):
    lista_gramas.append((palabra[i:i+q],1))
  return lista_gramas

In [45]:
# Contar ocurrencias de bigramas
plot_summaries_bigramas = plot_summaries_flat.flatMap(lambda x: gramas(x,2)) \
                                              .reduceByKey(lambda a, b: a + b) \
                                              .sortBy(lambda x: x[1], ascending = False)

for palabra in plot_summaries_bigramas.take(10):
  print(palabra)

22/02/28 22:39:35 WARN TaskSetManager: Stage 52 contains a task of very large size (18273 KiB). The maximum recommended task size is 1000 KiB.


('in', 888731)
('er', 778264)
('es', 680630)
('re', 585268)
('an', 562525)
('ng', 556631)
('ar', 514032)
('en', 512426)
('on', 505966)
('te', 484857)


In [46]:
# Cuenta de bigramas
plot_summaries_bigramas.count()

3433

In [47]:
# Ver qué bigramas se repiten menos

p = plot_summaries_bigramas.sortBy(lambda x: x[1], ascending = True)

for palabra in p.take(10):
  print(palabra)

('íq', 1)
('6¢', 1)
('’”', 1)
('而不', 1)
('不改', 1)
('是謂', 1)
('çã', 1)
('\u200bh', 1)
('\u200bs', 1)
('\n1', 1)


In [48]:
# Función para detectar gramas validos
import re

def valido_grama(grama, n):

  pattern = re.compile("[a-z]{" + str(n) + "}")
  if pattern.match(grama):
    return True

valido_grama('ac', 2)

True

In [49]:
# Bigramas validos menos repetidos
p = plot_summaries_bigramas.filter(lambda x : valido_grama(x[0],2)).sortBy(lambda x: x[1], ascending = True)

for palabra in p.take(10):
  print(palabra)

('bx', 1)
('fq', 2)
('jq', 2)
('wx', 2)
('vq', 2)
('qy', 3)
('qd', 3)
('qx', 3)
('cx', 3)
('qp', 4)


In [50]:
# Cuenta bigramas, esperados: 26^2=276
p.count()

673

In [51]:
# Contar ocurrencias por trigrama
plot_summaries_trigramas = plot_summaries_flat.flatMap(lambda x: gramas(x,3)). \
                                              reduceByKey(lambda a, b: a + b). \
                                              sortBy(lambda x: x[1], ascending = False)

for palabra in plot_summaries_trigramas.take(10):
  print(palabra)

22/02/28 22:42:41 WARN TaskSetManager: Stage 86 contains a task of very large size (18273 KiB). The maximum recommended task size is 1000 KiB.


('ing', 435139)
('ent', 160454)
('ter', 150150)
('her', 140996)
('ion', 134398)
('the', 125719)
('ers', 123246)
('ate', 114046)
('all', 112322)
('ver', 105528)


In [52]:
# Trigramas validos menos repetidos
p = plot_summaries_trigramas.filter(lambda x : valido_grama(x[0],3)).sortBy(lambda x: x[1], ascending = True)

for palabra in p.take(10):
  print(palabra)

('yqv', 1)
('ehj', 1)
('vty', 1)
('drw', 1)
('bph', 1)
('fgc', 1)
('tpj', 1)
('rfd', 1)
('pyx', 1)
('lnt', 1)


## Porcentaje de Peliculas que mencionan una Palabra


In [53]:
def pertenece(palabra, documento):
  if palabra in documento:
    return 1
  else: 
    return 0

In [54]:
def cuenta(palabra):
  plot_summaries_cuenta = plot_summaries_punct.map(lambda x: pertenece(palabra, x)).reduce(lambda a,b:a+b)

  print(plot_summaries_cuenta)

cuenta('father')

22/02/28 22:46:07 WARN TaskSetManager: Stage 103 contains a task of very large size (18273 KiB). The maximum recommended task size is 1000 KiB.


9084


In [55]:
cuenta('gay')

22/02/28 22:48:16 WARN TaskSetManager: Stage 104 contains a task of very large size (18273 KiB). The maximum recommended task size is 1000 KiB.


578


In [56]:
def porcentaje(palabra):
  plot_summaries_cuenta = plot_summaries_punct.map(lambda x: pertenece(palabra, x)).reduce(lambda a,b:a+b)

  print(str(plot_summaries_cuenta*100/plot_summaries_punct.count()))

porcentaje('father')

22/02/28 22:50:07 WARN TaskSetManager: Stage 105 contains a task of very large size (18273 KiB). The maximum recommended task size is 1000 KiB.
22/02/28 22:51:43 WARN TaskSetManager: Stage 106 contains a task of very large size (18273 KiB). The maximum recommended task size is 1000 KiB.


21.473654350755265


In [57]:
porcentaje('gay')

22/02/28 22:53:23 WARN TaskSetManager: Stage 107 contains a task of very large size (18273 KiB). The maximum recommended task size is 1000 KiB.
22/02/28 22:55:07 WARN TaskSetManager: Stage 108 contains a task of very large size (18273 KiB). The maximum recommended task size is 1000 KiB.


1.3663333569723188


## Términos Despectivos

In [71]:
cont_f = cont.map(lambda x: (x[0], x[1].count('faggot')))
cont_f = cont_f.sortBy(lambda renglon: renglon[1], ascending = False)
cont_f.take(5)

22/03/01 09:43:25 WARN TaskSetManager: Stage 123 contains a task of very large size (18273 KiB). The maximum recommended task size is 1000 KiB.
22/03/01 09:44:23 WARN TaskSetManager: Stage 124 contains a task of very large size (18273 KiB). The maximum recommended task size is 1000 KiB.
22/03/01 09:45:20 WARN TaskSetManager: Stage 125 contains a task of very large size (18273 KiB). The maximum recommended task size is 1000 KiB.


[('23753991', 3),
 ('5752544', 2),
 ('17997162', 1),
 ('12298570', 1),
 ('1834238', 1)]

In [72]:
character.filter(lambda x: x[0] == '23753991').take(1)

22/03/01 09:46:43 WARN TaskSetManager: Stage 127 contains a task of very large size (12858 KiB). The maximum recommended task size is 1000 KiB.
22/03/01 09:46:44 WARN TaskSetManager: Stage 128 contains a task of very large size (12866 KiB). The maximum recommended task size is 1000 KiB.


[['23753991',
  '/m/06zshz4',
  '2009',
  '',
  '',
  '',
  '',
  '',
  'Cameron Mitchell Mason',
  '',
  '/m/0gcjgpp',
  '',
  '/m/0gbz7bj']]

## Equidad (?)

In [73]:
waiter=cuenta('waiter')
waitress=cuenta('waitress')
mister=cuenta('mister')
miss=cuenta('miss')
actor=cuenta('actor')
actress=cuenta('actress')
businessman=cuenta('businessman')
businesswoman=cuenta('businesswoman')
girl=cuenta('girl')
boy=cuenta('boy')
policeman=cuenta('policeman')
policewoman=cuenta('policewoman')

22/03/01 09:53:04 WARN TaskSetManager: Stage 129 contains a task of very large size (18273 KiB). The maximum recommended task size is 1000 KiB.


196


22/03/01 09:54:21 WARN TaskSetManager: Stage 130 contains a task of very large size (18273 KiB). The maximum recommended task size is 1000 KiB.
22/03/01 09:55:17 WARN TaskSetManager: Stage 131 contains a task of very large size (18273 KiB). The maximum recommended task size is 1000 KiB.


352


36


...

22/03/01 09:58:55 WARN TaskSetManager: Stage 134 contains a task of very large size (18273 KiB). The maximum recommended task size is 1000 KiB.


768


ERROR:root:KeyboardInterrupt while sending command.                 (0 + 4) / 4]
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.9/site-packages/py4j/clientserver.py", line 475, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/local/Cellar/python@3.9/3.9.10/Frameworks/Python.framework/Versions/3.9/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

Exception in thread "serve RDD 169" java.net.SocketTimeoutException: Accept timed out
	at java.net.PlainSocketImpl.socketAccept(Native Method)
	at java.net.AbstractPlainSocketImpl.accept(AbstractPlainSocketImpl.java:535)
	at java.net.ServerSocket.implAccept(ServerSocket.java:545)
	at java.net.ServerSocket.accept(ServerSocket.java:513)
	at org.apache.spark.security.SocketAuthServer$$anon$1.run(SocketAuthServer.scala:64)


In [ ]:
cuenta('man')
cuenta('woman')

In [ ]:
cuenta('father')
cuenta('mother')

In [ ]:
cuenta('dad')
cuenta('mom')

In [ ]:
import matplotlib.pyplot as plt

eje_x = ['waiter', 'waitress', 'mister', 'miss', 'actor', 'actress', 'businessman', 'businesswoman', 'policeman', 'policewoman']
eje_y = [196,352,36,741,768,773,865,54,556,27]
 
plt.barh(eje_x, eje_y, color=['blue', 'pink'])
 
plt.xlabel('Cantidad de menciones en resumenes')
 
plt.ylabel('Palabras con genéro')
 
plt.title('Comparación entre masculino y femenino')
 
plt.show()

In [ ]:
cuenta('jesus')
cuenta('hugo')
cuenta('enrique')
cuenta('blanca')
cuenta('daniel')
cuenta('john')